In [32]:

from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
bsObj = BeautifulSoup(html)
print(bsObj.prettify())

tb = bsObj.find('table', class_='wikitable')
table_rows = tb.find_all('tr')
table_rows

In [8]:
pip install beautifulsoup4

     |████████████████████████████████| 102kB 4.6MB/s ta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install lxml

     |████████████████████████████████| 5.7MB 3.4MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
#Properly importing data in a DataFrame using Pandas
import pandas as pd
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
res
df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighbourhood"])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [16]:
#If a borough has not assigned , we will drop it 
dfClean = df[df.Borough != 'Not assigned']
dfClean.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Now we need to replace the Neighbourhood with Borough if Neighbourhood is not assigned

In [23]:
#Basically I found the location of the Neighbourhood with not assigned as value
locs=dfClean.loc[dfClean.Neighbourhood == 'Not assigned','Neighbourhood']
locs

Series([], Name: Neighbourhood, dtype: object)

In [22]:
#Here I replaced it with Queen's Park , Pretty lame method .. not fanciful. 
dfClean.Neighbourhood[8]=dfClean.Borough[8]
dfClean.Neighbourhood[8]

/srv/conda/envs/notebook/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


"Queen's Park"

In [28]:
#Here I used the agg method to join the Neighbourhood with same Borough and Post Code
df1 = dfClean.groupby(["Postcode", "Borough"],as_index=False,sort=False).agg(','.join)
df1.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"
